In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import math
%matplotlib inline

import sys
sys.path.insert(0, '../scripts')

import stft_zoom, display, detect_musical_regions
from util import *
import mappings
import pickle
import PIL
import IPython.display
from classes import SingleResSpectrogram, MultiResSpectrogram

/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [3]:
%load_ext line_profiler

In [4]:
def energy_reference(y, time_range, freq_range, sr=44100):
    y_slice = y[math.floor(time_range[0] * sr): math.floor(time_range[1] * sr)]
    spec = np.fft.rfft(y_slice)
    freqs = np.fft.rfftfreq(len(y_slice), 1./sr)
    idx_start = find_nearest(freqs, freq_range[0])
    idx_stop = find_nearest(freqs, freq_range[1])
    return np.sum(np.abs(spec[idx_start:idx_stop])**2) * 2 / len(y_slice)

def normalize_subregion(spec_zoom, time_range, freq_range, y):
    energy_ref = energy_reference(y, time_range, freq_range)
    energy_old = np.sum(spec_zoom ** 2)
    return math.sqrt(energy_ref / energy_old) * spec_zoom

In [5]:
model = pickle.load(open('renyi_shannon_prollharm_model.sav', 'rb'))
y, sr = librosa.load('../data/example.wav', sr=44100)

/Library/Python/3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GaussianNB from version 0.19.0 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Profiling por partes:

In [17]:
def initialize_data_struct():
    n_fft = 2048
    kernel = [800,800]
    spec = np.abs(librosa.stft(y, n_fft=n_fft))
    x_axis, y_axis = stft_zoom.get_axes_values(sr, 0, [0,len(y)/sr], spec.shape)

    base_spec = SingleResSpectrogram(spec, x_axis, y_axis)
    multires_spec = MultiResSpectrogram(base_spec)

In [18]:
%lprun -f initialize_data_struct initialize_data_struct()

In [19]:
## 56% para a stft, 43% para inicializar o SingleResSpec

In [20]:
n_fft = 2048
kernel = [800,800]
spec = np.abs(librosa.stft(y, n_fft=n_fft))
x_axis, y_axis = stft_zoom.get_axes_values(sr, 0, [0,len(y)/sr], spec.shape)

base_spec = SingleResSpectrogram(spec, x_axis, y_axis)
multires_spec = MultiResSpectrogram(base_spec)

In [21]:
kernel = [800,800]
threshold = 0.8
indices, shape_feature_map = detect_musical_regions.detect_musical_regions(model, spec, mode='threshold', pct_or_threshold=threshold)

In [22]:
%lprun -f detect_musical_regions.detect_musical_regions detect_musical_regions.detect_musical_regions(model, spec, mode='threshold', pct_or_threshold=0.8)

In [23]:
## 85% extraindo features

In [24]:
interesting_regions = detect_musical_regions.musical_regions_to_ranges(indices, shape_feature_map, x_axis, y_axis, kernel)

In [25]:
%lprun -f detect_musical_regions.musical_regions_to_ranges detect_musical_regions.musical_regions_to_ranges(indices, shape_feature_map, x_axis, y_axis, kernel)

In [26]:
# tempo desprezivel

In [27]:
subregion = interesting_regions[0]

freq_range = subregion[0]
time_range = subregion[1]
    
# Zoom in using "stft_zoom" (it prints out what subband processing method was applied)
spec_zoom, x_axis, y_axis, new_sr, window_size, hop_size = stft_zoom.stft_zoom(y, freq_range, time_range, sr=sr, original_window_size=n_fft, k=3)
    
# Each refined subregion is a SingleResSpectrogram of its own...
refined_subspec = SingleResSpectrogram(spec_zoom, x_axis, y_axis)
# ...that is inserted into a MultiResSpectrogram
multires_spec.insert_zoom(multires_spec.base_spec, refined_subspec, zoom_level=1)

ring mod + lpf


In [28]:
def zoom():
    freq_range = subregion[0]
    time_range = subregion[1]

    # Zoom in using "stft_zoom" (it prints out what subband processing method was applied)
    spec_zoom, x_axis, y_axis, new_sr, window_size, hop_size = stft_zoom.stft_zoom(y, freq_range, time_range, sr=sr, original_window_size=n_fft, k=3)

    # Each refined subregion is a SingleResSpectrogram of its own...
    refined_subspec = SingleResSpectrogram(spec_zoom, x_axis, y_axis)
    # ...that is inserted into a MultiResSpectrogram
    multires_spec.insert_zoom(multires_spec.base_spec, refined_subspec, zoom_level=1)

In [29]:
%lprun -f zoom zoom()

ring mod + lpf


In [30]:
%lprun -f stft_zoom.stft_zoom stft_zoom.stft_zoom(y, freq_range, time_range, sr=sr, original_window_size=n_fft, k=3)


ring mod + lpf


In [31]:
# 94% no filter_and_mod()

In [33]:
%lprun -f stft_zoom.filter_and_mod stft_zoom.filter_and_mod(y, freq_range, sr)

ring mod + lpf


In [35]:
# tempo dividido entre duas filtragens:

In [36]:
wp = np.array([freq_range[0] - 50, freq_range[1] + 50]) # lower freq. for bandpass filter 
ws = np.array([wp[0] - 50, wp[1] + 150]) # higher freq. for bandpass filter

new_sr = stft_zoom.find_undersample_fs(ws) # if new_sr, an undersampling frequency was found

wp = wp / (sr/2)
ws = ws / (sr/2)
    
y_filt = stft_zoom.filter_bandpass(y, wp, ws, sr) # bandpass filter the signal    

In [46]:
%lprun -f stft_zoom.filter_bandpass stft_zoom.filter_bandpass(y, wp, ws, sr)

In [39]:
# 30% p/ criar o filtro (buttord nao gasta nada), 70% para aplicar a filtragem
# (por isso talvez valha a pena pré filtrar)

In [44]:
new_sr = (ws[1] - ws[0] + 100/(sr/2)) * sr
mod = stft_zoom.ring_mod(y_filt, ws[0]*(sr/2), sr)
a = stft_zoom.filter_lowpass(mod, new_sr/2 - 100, sr), new_sr, ws[0]*(sr/2), False         

In [48]:
%lprun -f stft_zoom.ring_mod stft_zoom.ring_mod(y_filt, ws[0]*(sr/2), sr)

In [49]:
# 14 ms para fazer a modulaçao (para referencia, filtro bandpass gasta 18ms)
# ou seja, também vale a pena pré-modular

In [50]:
%lprun -f stft_zoom.filter_lowpass stft_zoom.filter_lowpass(mod, new_sr/2 - 100, sr)

In [51]:
# praticamente o mesmo tempo da filtragem e distribuicao (50% criar filtro, 50% filtrar)

Ou seja, acho que aumentaria bastante a performance pré-filtrar o sinal, modular pra perto de 0 Hz e filtrar com o lowpass nas faixas definidas pelo tamanho de kernel usado.

Criar funcao que recebe tamanho de kernel e cria banco de sinais correspondente (banco de filtros + mod + filtro de novo)

In [62]:
def make_signal_bank(y, kernel_size):
    fft_freqs = mappings.fft_frequencies(sr=sr, n_fft=n_fft)
    idx_list = mappings.find_freq_list(fft_freqs, kernel_size[1]) # essa linha:
    freq_list = fft_freqs[idx_list]
    y_bank = []
    freq_range = [0,0]
    
    # como funciona aindexacao? algo tipo.. y_bank[0] representa freqs entre freq_list[0-1]
    
    for i in range(len(freq_list)):
        freq_range[0] = freq_list[i]
        freq_range[1] = freq_list[i+1]

        if freq_range[0] <= 200:
            return filter_lowpass(y, freq_range[1], sr), 2*(freq_range[1]+100), 0, inverted
    

In [66]:
make_signal_bank(y, [500,500])

array([    0.        ,    21.53320312,    43.06640625,    64.59960938,
         107.66601562,   150.73242188,   215.33203125,   301.46484375,
         409.13085938,   559.86328125,   753.66210938,  1012.06054688,
        1356.59179688,  1830.32226562,  2454.78515625,  3294.58007812,
        4414.30664062,  5900.09765625,  7881.15234375, 10529.73632812,
       14061.18164062, 18776.953125  ])

In [ ]:
def filter_and_mod(y, freq_range, sr):

#   filter_and_mod() chooses between the following three options:
#         1) if freq_range[1] < 200, filter with low-pass and do not modulate
#         2) else, filter with band-pass and:
#            a) if possible, find an undersampling frequency. Check if the spectrum will be inverted by subsampling with the found frequency rate
#            b) else, perform ring-modulation and filter with low-pass (this is a cheaper version of SSB modulation)
#   Returns the filtered and modulated signal, the new sampling rate, the freq. mapped to DC and a boolean variable that
#   tells if the spectrum is inverted or not (it can happen with undersampling)

    inverted = False # if undersampling is performed with an even 'n', the spectrum is mirrored and will be unmirrored afterwards

    if freq_range[0] <= 200:
        return filter_lowpass(y, freq_range[1], sr), 2*(freq_range[1]+100), 0, inverted
      
    wp = np.array([freq_range[0] - 50, freq_range[1] + 50]) # lower freq. for bandpass filter 
    ws = np.array([wp[0] - 50, wp[1] + 150]) # higher freq. for bandpass filter

    new_sr = find_undersample_fs(ws) # if new_sr, an undersampling frequency was found

    wp = wp / (sr/2)
    ws = ws / (sr/2)
    
    y_filt = filter_bandpass(y, wp, ws, sr) # bandpass filter the signal
    
    if not new_sr: # if undersampling is not possible, perform ringmod + lpf
        new_sr = (ws[1] - ws[0] + 100/(sr/2)) * sr
        print("ring mod + lpf")
        return filter_lowpass(ring_mod(y_filt, ws[0]*(sr/2), sr), new_sr/2 - 100, sr), new_sr, ws[0]*(sr/2), inverted         
        